This notebook checks if the pytorch lighning early stopping implementation loads back the best weight after the trining has stopped.

In [1]:
from train import *

In [2]:
train_dl = th.utils.data.DataLoader(
    train_ds, batch_size=hyp_params.batch_size, pin_memory=True, num_workers=1
)
valid_dl = th.utils.data.DataLoader(
    valid_ds, batch_size=hyp_params.batch_size, pin_memory=True, num_workers=1
)

In [3]:
import pandas as pd

import mutils

df = pd.read_csv(mutils.csv_dir / "6c6aea26edaa41429b337f389415fcc2" / "version_0" / "metrics.csv")
df = mutils.get_epoch_info(df)

In [29]:
import pytorch_lightning as pl
import torch as th
from pytorch_lightning.metrics import MeanAbsoluteError
from torch.nn import functional as F

from models import MULTModel
from loss import bell_loss, bell_mse_mae_loss

loss_dict = {"L2": F.mse_loss, "Bell": bell_loss, "BellL1L2": bell_mse_mae_loss}


class MULTModelWarped(pl.LightningModule):
    def __init__(self, hyp_params, target_names, data):
        super().__init__()
        self.model = MULTModel(hyp_params)
        self.save_hyperparameters(hyp_params)
        self.learning_rate = hyp_params.lr
        self.target_names = target_names

        self.mae_1 = 1 - MeanAbsoluteError()
        self.loss = loss_dict[hyp_params.loss_fnc]
        
        self.data = data
        self.validation_step_i = 0
        
    def _fake_metric(self):
        return {'valid_1mae': self.data[self.current_epoch]}

    def forward(self, *args):
        if len(args) == 3:
            text, audio, face = args
        else:
            text, audio, face = args[0]
        return self.model(text, audio, face)[0]

    def configure_optimizers(self):
        optimizer = th.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def training_step(self, batch, batch_idx):
        metric_values = self._calc_loss_metrics(batch)
        metric_values = {f"train_{k}": v for k, v in metric_values.items()}
#         self.log_dict(
#             metric_values, on_step=True, on_epoch=True, prog_bar=False, logger=True
#         )
#         print(early_stopping.wait_count)
        return metric_values["train_loss"]

    def validation_step(self, batch, batch_idx):
#         metric_values = self._calc_loss_metrics(batch)
#         metric_values = {f"valid_{k}": v for k, v in metric_values.items()}
        metric_values = {'valid_1mae': self.data[self.validation_step_i]}
#         metric_values['valid_1mae'] = self.data[self.validation_step_i]
        self.validation_step_i += 1
        self.log_dict(metric_values, prog_bar=False, logger=True)
        return metric_values

#     def validation_epoch_end(self, validation_step_outputs):
#         pass
#         print(len(validation_step_outputs))

    def test_step(self, batch, batch_idx):
        metric_values = self._calc_loss_metrics(batch)
        metric_values = {f"test_{k}": v for k, v in metric_values.items()}
        self.log_dict(metric_values, prog_bar=False, logger=True)
        return metric_values

    def _calc_loss_metrics(self, batch):
        audio, face, text, y = batch
        y_hat = self(text, audio, face)
        loss = self.loss(y_hat, y)
        metric_values = self._calc_mae1_columnwise(y_hat, y)
        metric_values["loss"] = loss
        metric_values["1mae"] = self.mae_1(y_hat, y)
        return metric_values

    def _calc_mae1_columnwise(self, y_hat, y):
        return {
            f"1mae_{name}": self.mae_1(y_hat[:, i], y[:, i])
            for i, name in enumerate(self.target_names)
        }

In [41]:
class MULTModelWarped2(MULTModelWarped):
    pass
#     def validation_epoch_end(self, validation_step_outputs):
#         pass

In [42]:
accs = [0,1,3,2,4] #df['valid_1mae'].to_list()
model = MULTModelWarped2(hyp_params, target_names, accs)

In [43]:
early_stopping = EarlyStopping(monitor="valid_1mae", patience=10, verbose=True, mode="max")
trainer = pl.Trainer(
    gpus=1,
    max_epochs=1,#hyp_params.num_epochs,
    log_every_n_steps=1,
    callbacks=[EarlyStopping(monitor="valid_1mae", patience=10, verbose=True, mode="max")],
    limit_train_batches=1,
    limit_val_batches=2,
)
trainer.fit(model, train_dl, valid_dl)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name  | Type                | Params
----------------------------------------------
0 | model | MULTModel           | 1.1 M 
1 | mae_1 | CompositionalMetric | 0     
----------------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.405     Total estimated model params size (MB)


1

In [44]:
trainer.callback_metrics

{'valid_1mae': tensor(2., device='cuda:0')}